# Parsing the cards information

In [ ]:
using System.Text.RegularExpressions;
using System.IO;

string text = File.ReadAllText(@"munchkin.deluxe.card.description.txt");
var sections = text.Split(new string[] { Environment.NewLine + Environment.NewLine }, StringSplitOptions.RemoveEmptyEntries);

In [ ]:
sections[1]

Class Cards
	3 Thief: Backstabbing - You may discard a card to backstab another player (-2 in combat). You may do this only once per victim per combat, but if two players are fighting a monster together, you may backstab each of them. Theft - You may discard a card to try to steal a small item carried by another player. Roll a die; 4 or more succeeds. Otherwise, you get whacked and lose 1 Level.
	3 Wizard: Flight Spell - You may discard up to 3 cards while Running Away; each one gives you a +1 bonus to flee. Charm Spell - You may discard your whole hand to charm a single Monster instead of fighting it. Discard the Monster and take its Treasure, but don't gain levels. If there are other monsters in the combat, fight them normally.
	3 Warrior: Berserking - You may discard up to 3 cards in combat; each one gives you a +1 bonus. You win ties in combat.
	3 Cleric: Resurrection - When it is time for you to draw a card face-up, you may instead take the top card from the appropriate discard pi

In [ ]:
var specialPattern = @"^\t((?<count>\d) (?<title>.*?): ((?<special>.*?) - )?(?<desc>.*?))$";

In [ ]:
void PrintLast5(MatchCollection matches)
{
    foreach(Match match in matches.TakeLast(5))
        Console.WriteLine(match.Groups["title"].Value);
}

## Race Cards

In [ ]:
var raceMatches = Regex.Matches(sections[0], specialPattern, RegexOptions.Multiline);
PrintLast5(raceMatches)

Dwarf


Elf


Halfing


Half-Breed


## Class Cards

In [ ]:
var classMatches = Regex.Matches(sections[1], specialPattern, RegexOptions.Multiline);
PrintLast5(classMatches);

Thief


Wizard


Warrior


Cleric


Super Munchkin


## Curse Cards

In [ ]:
var curseMatches = Regex.Matches(sections[2], specialPattern, RegexOptions.Multiline);
PrintLast5(curseMatches);

Chicken On Your Head


Truly Obnoxious Curse


Lose Your Race


Duck of Doom


Lose Two Cards


## Special Cards

In [ ]:
var specialMatches = Regex.Matches(sections[3], specialPattern, RegexOptions.Multiline);
PrintLast5(specialMatches);

Help Me Out Here!


Out To Lunch


Mate


Wandering Monster


Cheat!


## Monster Cards

In [ ]:
string monsterPattern = @"^\t((?<title>.*?) - L(?<power>\d+) T(?<treasures>\d+)( (?<levels>\d+)L)? - (?<desc>.*?))$";
var monsterMatches = Regex.Matches(sections[4], monsterPattern, RegexOptions.Multiline);
PrintLast5(monsterMatches);

King Tut (Undead)


Wight Brothers (Undead)


Squidzilla


Bullrog


Plutonium Dragon


## Putting it all together

Defining the common types for parsed data:

In [ ]:
public record CardInfo(
    string Type,
    string Title,
    string Description,
    int MonsterLevel,
    int RewardTreasures,
    int RewardLevels,
    string SpecialSubtitle
);

In [ ]:
int ParseOrDefault(string value)
{
    return int.TryParse(value, out int number) ? number : 0;
}

Converting the data into actual types:

In [ ]:
var raceCardsInfo = raceMatches
    .Select(x => new CardInfo(
        "Race",
        x.Groups["title"].Value,
        x.Groups["desc"].Value,
        ParseOrDefault(x.Groups["power"].Value),
        ParseOrDefault(x.Groups["treasures"].Value),
        ParseOrDefault(x.Groups["levels"].Value),
        x.Groups["special"].Value));

In [ ]:
var classCardsInfo = classMatches
    .Select(x => new CardInfo(
        "Class",
        x.Groups["title"].Value,
        x.Groups["desc"].Value,
        ParseOrDefault(x.Groups["power"].Value),
        ParseOrDefault(x.Groups["treasures"].Value),
        ParseOrDefault(x.Groups["levels"].Value),
        x.Groups["special"].Value));

In [ ]:
var curseCardsInfo = curseMatches
    .Select(x => new CardInfo(
        "Curse",
        x.Groups["title"].Value,
        x.Groups["desc"].Value,
        ParseOrDefault(x.Groups["power"].Value),
        ParseOrDefault(x.Groups["treasures"].Value),
        ParseOrDefault(x.Groups["levels"].Value),
        x.Groups["special"].Value));

In [ ]:
var specialCardsInfo = specialMatches
    .Select(x => new CardInfo(
        "Special",
        x.Groups["title"].Value,
        x.Groups["desc"].Value,
        ParseOrDefault(x.Groups["power"].Value),
        ParseOrDefault(x.Groups["treasures"].Value),
        ParseOrDefault(x.Groups["levels"].Value),
        x.Groups["special"].Value));

In [ ]:
var monsterCardsInfo = monsterMatches
    .Select(x => new CardInfo(
        "Curse",
        x.Groups["title"].Value,
        x.Groups["desc"].Value,
        ParseOrDefault(x.Groups["power"].Value),
        ParseOrDefault(x.Groups["treasures"].Value),
        ParseOrDefault(x.Groups["levels"].Value),
        x.Groups["special"].Value));

In [ ]:
var cardsInfo = raceCardsInfo
    .Concat(classCardsInfo)
    .Concat(curseCardsInfo)
    .Concat(specialCardsInfo)
    .Concat(monsterCardsInfo);

foreach(var info in cardsInfo.TakeLast(5))
    Console.WriteLine(info.Title);

King Tut (Undead)


Wight Brothers (Undead)


Squidzilla


Bullrog


Plutonium Dragon
